In [4]:
import numpy as np


def get_value_for_period(time, array):
    """
    Given a point in time (in years), returns the appropriate value for the period 
    
    Parameters:
    time: float 
        time in years
    array: np.array 
        list of values for each 6 months
    """
    return array[int(time*2) if int(time*2)<6 else 5] # works


def correlated_path_generator_3d(S0_A, S0_B, S0_C, T, rates, yields, volatilites, corr_matrix, N, Ni, kappa, Notional):
    """
    Generates Ni random paths for two correlated underlying asset prices S_t1 and S_t2 using a log-normal process.

    Parameters:
    S0_A, S0_B, S0_C : float
        Initial stock prices
    T : float
        Time to maturity (in years)
    r : float
        Risk-free interest rate (annual)
    q_A, q_B, q_C : float
        Dividend yields (annual) for each stock
    sigma_A, sigma_B, sigma_C : float
        Volatilities of the underlying assets
    corr_matrix : float
        Correlation matrix between the three assets
    N : int
        Number of time steps
    Ni : int
        Number of paths to generate

    Returns:
    tuple of numpy.ndarray
        Arrays of shape (Ni, N+1) containing Ni random paths for S_tA, S_tB and S_tC
    """
    dt = T / N
    paths_A = np.zeros((Ni, N+1))
    paths_B = np.zeros((Ni, N+1))
    paths_C = np.zeros((Ni, N+1))

    paths_A[:, 0] = S0_A
    paths_B[:, 0] = S0_B
    paths_C[:, 0] = S0_C
    
    # Cholesky decomposition to generate correlated random variables
    A = np.linalg.cholesky(corr_matrix)

    observation_dates = [int(N*i/10) for i in range(1, 11)] #  number of days*i/9
    # print(observation_dates)

    final_undiscounted_payoff = np.zeros(Ni)
    exercise_time = np.zeros(Ni)
    
    for i in range(1, N+1):
        Z = np.random.normal(0, 1, (Ni, 3))
        Z_corr = Z.dot(A.T)  # Apply the Cholesky matrix to get correlated random variables
        paths_A[:, i] = paths_A[:, i-1] * np.exp((get_value_for_period(i*dt, rates) - get_value_for_period(i*dt, yields[0]) - 0.5 * get_value_for_period(i*dt, volatilites[0])**2) * dt 
                                                 + np.sqrt(dt) * get_value_for_period(i*dt, volatilites[0]) * Z_corr[:, 0])
        paths_B[:, i] = paths_B[:, i-1] * np.exp((get_value_for_period(i*dt, rates) - get_value_for_period(i*dt, yields[1]) - 0.5 * get_value_for_period(i*dt, volatilites[1])**2) * dt 
                                                 + np.sqrt(dt) * get_value_for_period(i*dt, volatilites[1]) * Z_corr[:, 1])
        paths_C[:, i] = paths_C[:, i-1] * np.exp((get_value_for_period(i*dt, rates) - get_value_for_period(i*dt, yields[2]) - 0.5 * get_value_for_period(i*dt, volatilites[2])**2) * dt 
                                                 + np.sqrt(dt) * get_value_for_period(i*dt, volatilites[2]) * Z_corr[:, 2])
        
        if i in observation_dates and i!=N:
            # print(i)
            for path in range(0, Ni):
                relative_performance_A = paths_A[path, i]/S0_A
                relative_performance_B = paths_B[path, i]/S0_B
                relative_performance_C = paths_C[path, i]/S0_C
                best_relative_performance = np.maximum(np.maximum(relative_performance_A, relative_performance_B), relative_performance_C)
                if final_undiscounted_payoff[path] == 0 and best_relative_performance >= kappa:
                    final_undiscounted_payoff[path] = Notional + 0.07*Notional*i*10/N
                    exercise_time[path] = i
                # print(exercise_time[path], final_undiscounted_payoff[path])
        elif i == N:
            for path in range(0, Ni):
                relative_performance_A = paths_A[path, i]/S0_A
                relative_performance_B = paths_B[path, i]/S0_B
                relative_performance_C = paths_C[path, i]/S0_C
                best_relative_performance = np.maximum(np.maximum(relative_performance_A, relative_performance_B), relative_performance_C) 
                if final_undiscounted_payoff[path] == 0: 
                    exercise_time[path] = i
                    if best_relative_performance >= kappa:
                        final_undiscounted_payoff[path] = 1.15*Notional
                    elif best_relative_performance < kappa and best_relative_performance > 0.6:
                        final_undiscounted_payoff[path] = Notional
                    else: 
                        final_undiscounted_payoff[path] = Notional*best_relative_performance
    # for path in range(Ni):
    #     print(exercise_time[path], final_undiscounted_payoff[path])
    return final_undiscounted_payoff, exercise_time


In [1]:
def discount(undiscounted_payoff, exercise_time, interest_rates, N, Ni):
    """
    undscounted_payoff: numpy.ndarray
        list of the final undiscounted payoff for each path 
    exercise_time: numpy.ndarray
        list of the exercise times for each path 
    interest_rates: numpy.ndarray 
        list of interest rates for each period
    N: int
        number of time steps for the whole period
    Ni: int
        number of paths
    """
    pv = np.zeros(Ni)

    for path in range(Ni):
        # for each path 
        # discounted = e^{-\int_0^T r(t)dt} P_undiscounted = e^{-0.5 \sigma_{i=0}^T r(t)}
        pv[path] = np.exp(-sum(0.5*interest_rates[:(int(6*exercise_time[path]/N) if int(6*exercise_time[path]/N) < 6 else 5)])) * undiscounted_payoff[path]
    return pv 

In [5]:
# Parameters based on pdf
S0_A, S0_B, S0_C = 150, 250, 300  # Initial stock prices
T = 5  # Time to maturity in years
interest_rates = np.array((0.015, 0.0175, 0.02, 0.021, 0.02, 0.019))
volatilities = np.array([[0.20, 0.21, 0.20, 0.19, 0.18, 0.19],  # Stock A
                         [0.22, 0.23, 0.22, 0.21, 0.20, 0.21],  # Stock B
                         [0.25, 0.26, 0.24, 0.23, 0.22, 0.23]]) # Stock C
interest_rates = np.array([0.015, 0.0175, 0.02, 0.021, 0.02, 0.019])
dividend_yields = np.array([[0.01, 0.015, 0.02, 0.019, 0.018, 0.017],  # Stock A
                            [0.015, 0.02, 0.025, 0.023, 0.021, 0.02],  # Stock B
                            [0.02, 0.025, 0.03, 0.025, 0.023, 0.022]]) # Stock C
r = 0.02  # Risk-free interest rate
q_A, q_B, q_C = 0.015, 0.02, 0.025  # Dividend yields
sigma_A, sigma_B, sigma_C = 0.2, 0.22, 0.25  # Volatilities
corr_matrix = np.array([[1, 0.75, 0.85], [0.75, 1, 0.9], [0.85, 0.9, 1]])
N = 261 * T  # Number of time steps (# of weekdays)
Ni = 10000  # Number of paths
Notional = 1000000
kappa = 1.15

# Test the function with example parameters
final_undiscounted_price, exercise_time = correlated_path_generator_3d(S0_A, S0_B, S0_C, T, interest_rates, dividend_yields, volatilities, corr_matrix, N, Ni, kappa, Notional)

# presdent value of the note (i.e., discounted)
pv = discount(final_undiscounted_price, exercise_time, interest_rates, N, Ni)

# print(f"Mean Undiscounted Price of the Note: ${final_undiscounted_price.mean():.2f}")
print(f"Mean Discounted Price of the Note: ${pv.mean():.2f}")
print(f"Standard Error: ${np.std(pv) / np.sqrt(len(pv)):.2f}")


Mean Discounted Price of the Note: $1072657.93
Standard Error: $2315.57
